## Content-based рекомендации для фильмов

Если грубо, суть Content-Based рекомендаций сводится к следующему: мы хотим рекомендовать товары, наиболее близкие к тем, что клиент уже покупал и оценил положительно.

In [44]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

Для нашего игрового примера загрузим описания нескольких фильмов. Обратите внимание, что метод pd.read_csv() умеет читать по прямой ссылке из интернета.

In [36]:
df = pd.read_csv('https://query.data.world/s/uikepcpffyo2nhig52xxeevdialfl7')
df.head(2)

,Unnamed: 0,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,...,tomatoConsensus,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,tomatoURL,DVD,BoxOffice,Production,Website,Response
0,1,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,"Crime, Drama",Frank Darabont,"Stephen King (short story ""Rita Hayworth and S...","Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",...,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/shawshank_rede...,27 Jan 1998,NaN,Columbia Pictures,NaN,True
1,2,The Godfather,1972,R,24 Mar 1972,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo (screenplay), Francis Ford Coppola ...","Marlon Brando, Al Pacino, James Caan, Richard ...",...,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather/,09 Oct 2001,NaN,Paramount Pictures,http://www.thegodfather.com,True


In [5]:
df.shape

(250, 38)

Для простоты будем использовать только некоторые фичи

In [37]:
df = df[['Title','Genre','Director','Actors','Plot']]
df.head()

,Title,Genre,Director,Actors,Plot
0,The Shawshank Redemption,"Crime, Drama",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...
1,The Godfather,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...
2,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...
3,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...
4,12 Angry Men,"Crime, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...


In [22]:
df.shape

(250, 5)

Немного предобработаем данные.
- Колонки "Актеры", "Жанр", "Режиссер" представим в виде списков (потом эти списки будем объединять)
- Из длинного списка актеров оставим только главных трех
- Имена склеим, так как имя здесь - это неделимый кусок информации

In [38]:
# discarding the commas between the actors' full names and getting only the first three names
df['Actors'] = df['Actors'].map(lambda x: x.split(',')[:3])

# putting the genres in a list of words
df['Genre'] = df['Genre'].map(lambda x: x.lower().split(','))

df['Director'] = df['Director'].map(lambda x: x.split(' '))

df['Plot'] = df['Plot'].map(lambda x: x.split(' '))

for index, row in df.iterrows():
    row['Actors'] = [x.lower().replace(' ','') for x in row['Actors']]
    row['Director'] = ''.join(row['Director']).lower()

Название фильма сделаем идентификатором в нашем датасете. Потом по нему сможем достатавть необходимые данные

In [39]:
df.set_index('Title', inplace = True)
df.head()

,Genre,Director,Actors,Plot
Title,,,,
The Shawshank Redemption,"[crime, drama]",frankdarabont,"[timrobbins, morganfreeman, bobgunton]","[Two, imprisoned, men, bond, over, a, number, ..."
The Godfather,"[crime, drama]",francisfordcoppola,"[marlonbrando, alpacino, jamescaan]","[The, aging, patriarch, of, an, organized, cri..."
The Godfather: Part II,"[crime, drama]",francisfordcoppola,"[alpacino, robertduvall, dianekeaton]","[The, early, life, and, career, of, Vito, Corl..."
The Dark Knight,"[action, crime, drama]",christophernolan,"[christianbale, heathledger, aaroneckhart]","[When, the, menace, known, as, the, Joker, eme..."
12 Angry Men,"[crime, drama]",sidneylumet,"[martinbalsam, johnfiedler, leej.cobb]","[A, jury, holdout, attempts, to, prevent, a, m..."


Объединим все описания (жанр, режиссер, актеры, сюжет) в единый текст (bag_of_words)

In [40]:
df['bag_of_words'] = ''
columns = df.columns
for index, row in df.iterrows():
    words = ''
    for col in columns:
        if col != 'Director':
            words = words + ' '.join(row[col])+ ' '
        else:
            words = words + row[col]+ ' '
    row['bag_of_words'] = words
    
df.drop(columns = [col for col in df.columns if col!= 'bag_of_words'], inplace = True)

In [41]:
df.head()

,bag_of_words
Title,
The Shawshank Redemption,crime drama frankdarabont timrobbins morganfr...
The Godfather,crime drama francisfordcoppola marlonbrando a...
The Godfather: Part II,crime drama francisfordcoppola alpacino rober...
The Dark Knight,action crime drama christophernolan christia...
12 Angry Men,crime drama sidneylumet martinbalsam johnfied...


In [42]:
df.bag_of_words[0]

'crime  drama frankdarabont timrobbins morganfreeman bobgunton Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.  '

Чтобы дальше работать с текстовыми описаниями фильмов, их нужно перевести в числовой формат (векторизовать). Для этого используем хорошо нам знакомый CountVectorizer()

In [14]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df['bag_of_words'])

# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use later to match the indexes
indices = pd.Series(df.index)
indices[:5]

0    The Shawshank Redemption
1               The Godfather
2      The Godfather: Part II
3             The Dark Knight
4                12 Angry Men
Name: Title, dtype: object

Из лекций по NLP мы хорошо помним, что одна из наиболее часто используемых метрик при вычислении расстояния между текстами - косинусное расстояние

$similarity = cos(\theta) = \dfrac{\boldsymbol{u} \cdot \boldsymbol{v}}{\|{\boldsymbol{u}}\|\|{\boldsymbol{v}}\|} = 
\dfrac{\sum_{i = 1}^n u_iv_i}{\sqrt{\sum_{i = 1}^n u_i^2}\sqrt{\sum_{i = 1}^n v_i^2}}$

In [15]:
# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[ 1.        ,  0.33333333,  0.33333333, ...,  0.15430335,
         0.15430335,  0.16666667],
       [ 0.33333333,  1.        ,  0.66666667, ...,  0.15430335,
         0.15430335,  0.16666667],
       [ 0.33333333,  0.66666667,  1.        , ...,  0.15430335,
         0.15430335,  0.16666667],
       ..., 
       [ 0.15430335,  0.15430335,  0.15430335, ...,  1.        ,
         0.14285714,  0.15430335],
       [ 0.15430335,  0.15430335,  0.15430335, ...,  0.14285714,
         1.        ,  0.15430335],
       [ 0.16666667,  0.16666667,  0.16666667, ...,  0.15430335,
         0.15430335,  1.        ]])

Считаем, что пользователь посмотрел некий фильм, положительно его оценил, и теперь мы будем искать фильмы, наиболее похожие по своему описанию на данный фильм.

Для этого на матрице близости (cosine_sim) выделим для выбранного фильма Top-10 фильмов с наибольшей similarity

In [16]:
# function that takes in movie title as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cosine_sim):
    
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(df.index)[i])
        
    return recommended_movies

К примеру рекомендации для фильма "Фарго"

In [17]:
recommendations('Fargo')

['No Country for Old Men',
 'The Big Lebowski',
 'Rope',
 'Reservoir Dogs',
 'Léon: The Professional',
 'The Departed',
 'On the Waterfront',
 'The Silence of the Lambs',
 'Casino',
 'To Kill a Mockingbird']

Мы считали, что клиент посмотрел только один фильм. Как быть, если их несколько? Просто суммируем описания фильмов в один вектор интересов.

**Упражнение**. Добавьте TF-IDF взвешивание при определении наиболее близких фильтмов.